# Introduction To Astro-Data Analysis

In this notebook, I will cover the key methods required to get started with processing astrophysical datasets.

We will cover:
- Downloading datasets directly from the source using scripts
- Dealing with different file formats
- Loading in data effectively
- Processing and manipulating data
- Visulation
- Storing Data
